# Spark Exercise

Apache Spark is an excellent tool for data engineering projects due to its robust ability to process large-scale data efficiently through distributed computing. Spark's in-memory processing capabilities significantly enhance the speed of data operations, making it ideal for handling big data workloads. It supports various data sources and formats, offering versatility in data ingestion and transformation. Additionally, Spark's rich API supports multiple programming languages such as Python, Java, and Scala, catering to diverse developer preferences. Its ecosystem, which includes libraries for SQL, machine learning, and graph processing, provides a comprehensive suite for building complex data pipelines and analytics, making it a powerful and flexible choice for data engineering tasks.

Use Python, ```pyspark``` and ```pandas``` to explore Apache Spark RDD and DataFrame:

# Spark RDD

Spark RDD (Resilient Distributed Dataset) is a fundamental data structure in Apache Spark that enables fault-tolerant, distributed processing of large datasets across multiple nodes in a cluster. Spark RDDs provide a higher-level abstraction for performing distributed data processing tasks, including both map (transformations) and reduce (aggregations) operations.

## Import Necessary Libraries

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pandas as pd

## Spark Context and Session
Initialize Spark Context and Spark Session

In [2]:
spark = (
    SparkSession
    .builder
    .appName("LoadClosingOddsRDD")
    .config("spark.master", "local[*]")
    .getOrCreate()
)

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/26 16:10:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/26 16:10:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/26 16:10:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/26 16:10:08 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/05/26 16:10:08 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


## Load Data into RDD

In [3]:
csv_path = "closing_odds.csv"
df = pd.read_csv(
    csv_path,
    compression="gzip",      
    encoding="utf-8",        
)

data_tuples = list(df.itertuples(index=False, name=None))

rdd = sc.parallelize(data_tuples)          

## Map Operation

Split data into individual parts and create key-value pairs

In [4]:
rdd_kv = rdd.map(lambda tup: (tup[0], tup[1:]))
print("key-value pairs:")
for kv in rdd_kv.take(5):
    print(kv)

key-value pairs:


25/05/26 16:13:30 WARN TaskSetManager: Stage 0 contains a task of very large size (13042 KiB). The maximum recommended task size is 1000 KiB.


(170088, ('England: Premier League', '2005-01-01', 'Liverpool', 0, 'Chelsea', 1, 2.9944, 3.1944, 2.2256, 3.2, 3.25, 2.29, 'Paddy Power', 'Sportingbet', 'Expekt', 9, 9, 9))
(170089, ('England: Premier League', '2005-01-01', 'Fulham', 3, 'Crystal Palace', 1, 1.9456, 3.2333, 3.6722, 2.04, 3.3, 4.15, 'Pinnacle Sports', 'bet-at-home', 'Expekt', 9, 9, 9))
(170090, ('England: Premier League', '2005-01-01', 'Aston Villa', 1, 'Blackburn', 0, 1.8522, 3.2611, 4.0144, 2.0, 3.4, 4.5, 'Pinnacle Sports', 'Paddy Power', 'Sportingbet', 9, 9, 9))
(170091, ('England: Premier League', '2005-01-01', 'Bolton', 1, 'West Brom', 1, 1.6122, 3.4133, 5.4722, 1.67, 3.57, 6.27, 'Coral', 'Pinnacle Sports', 'Pinnacle Sports', 9, 9, 9))
(170092, ('England: Premier League', '2005-01-01', 'Charlton', 1, 'Arsenal', 3, 5.9878, 3.4778, 1.5567, 7.0, 3.6, 1.62, 'Expekt', 'Paddy Power', 'bet365', 9, 9, 9))


## Reduce Operation

Reduce your key-value pairs

In [5]:
matches_per_league_rdd = (
    rdd_kv
    .map(lambda parts: (parts[1], 1))            
    .reduceByKey(lambda a, b: a + b)            
)

for league, count in matches_per_league_rdd.take(10):
    print(f"{league}: {count} games")

25/05/26 16:13:41 WARN TaskSetManager: Stage 1 contains a task of very large size (13042 KiB). The maximum recommended task size is 1000 KiB.


('England: Premier League', '2005-01-01', 'Aston Villa', 1, 'Blackburn', 0, 1.8522, 3.2611, 4.0144, 2.0, 3.4, 4.5, 'Pinnacle Sports', 'Paddy Power', 'Sportingbet', 9, 9, 9): 1 games
('England: Championship', '2005-01-01', 'Derby', 0, 'Cardiff', 1, 1.6875, 3.3625, 4.5188, 1.73, 3.5, 5.5, 'bet365', 'bet365', 'Coral', 8, 8, 8): 1 games
('England: Championship', '2005-01-01', 'QPR', 0, 'Brighton', 0, 1.8388, 3.2888, 3.7788, 1.9, 3.66, 4.03, 'Expekt', 'Pinnacle Sports', 'Pinnacle Sports', 8, 8, 8): 1 games
('England: Championship', '2005-01-01', 'Sheffield Utd', 0, 'Wigan', 2, 2.25, 3.2188, 2.7888, 2.4, 3.25, 3.0, 'Expekt', 'bet365', 'Ladbrokes', 8, 8, 8): 1 games
('England: Championship', '2005-01-01', 'Wolves', 1, 'Plymouth', 1, 1.8713, 3.2625, 3.675, 2.0, 3.4, 4.0, 'Pinnacle Sports', 'Pinnacle Sports', 'Sportingbet', 8, 8, 8): 1 games
('England: League One', '2005-01-01', 'Bristol City', 2, 'Peterborough', 0, 1.5378, 3.48, 5.6656, 1.57, 3.6, 6.54, 'bet365', 'bet365', 'Pinnacle Sports', 9

## Collect Results

Because of lazy evaluation, the map-reduce operation is performed only now. Show what you calculated.

In [6]:
matches_per_league_rdd.collect()

[(('England: Premier League',
   '2005-01-01',
   'Aston Villa',
   1,
   'Blackburn',
   0,
   1.8522,
   3.2611,
   4.0144,
   2.0,
   3.4,
   4.5,
   'Pinnacle Sports',
   'Paddy Power',
   'Sportingbet',
   9,
   9,
   9),
  1),
 (('England: Championship',
   '2005-01-01',
   'Derby',
   0,
   'Cardiff',
   1,
   1.6875,
   3.3625,
   4.5188,
   1.73,
   3.5,
   5.5,
   'bet365',
   'bet365',
   'Coral',
   8,
   8,
   8),
  1),
 (('England: Championship',
   '2005-01-01',
   'QPR',
   0,
   'Brighton',
   0,
   1.8388,
   3.2888,
   3.7788,
   1.9,
   3.66,
   4.03,
   'Expekt',
   'Pinnacle Sports',
   'Pinnacle Sports',
   8,
   8,
   8),
  1),
 (('England: Championship',
   '2005-01-01',
   'Sheffield Utd',
   0,
   'Wigan',
   2,
   2.25,
   3.2188,
   2.7888,
   2.4,
   3.25,
   3.0,
   'Expekt',
   'bet365',
   'Ladbrokes',
   8,
   8,
   8),
  1),
 (('England: Championship',
   '2005-01-01',
   'Wolves',
   1,
   'Plymouth',
   1,
   1.8713,
   3.2625,
   3.675,
   2.0,
   

## Save Results

In [8]:
matches_per_league_rdd \
    .map(lambda kv: f"{kv[0]},{kv[1]}") \
    .saveAsTextFile("./matches_per_league_txt")

# Spark DataFrame

Spark DataFrame is a distributed collection of data organized into named columns, designed for efficient data manipulation and analysis in Apache Spark. It is used for various data processing tasks such as data ingestion, transformation, querying, and analysis in Apache Spark, providing a high-level abstraction that simplifies working with structured data.

## Load Data into DataFrame

In [16]:
df = spark.read.format("csv") \
    .option("header",       "true") \
    .option("inferSchema",  "true") \
    .option("compression",  "gzip") \
    .load("./closing_odds.csv.gz")

## View DataFrame Schema

In [17]:
df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- league: string (nullable = true)
 |-- match_date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_team: string (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- avg_odds_home_win: double (nullable = true)
 |-- avg_odds_draw: double (nullable = true)
 |-- avg_odds_away_win: double (nullable = true)
 |-- max_odds_home_win: double (nullable = true)
 |-- max_odds_draw: double (nullable = true)
 |-- max_odds_away_win: double (nullable = true)
 |-- top_bookie_home_win: string (nullable = true)
 |-- top_bookie_draw: string (nullable = true)
 |-- top_bookie_away_win: string (nullable = true)
 |-- n_odds_home_win: integer (nullable = true)
 |-- n_odds_draw: integer (nullable = true)
 |-- n_odds_away_win: integer (nullable = true)



## View DataFrame Data

In [18]:
df.show(10)

+--------+--------------------+----------+---------------+----------+-----------------+----------+-----------------+-------------+-----------------+-----------------+-------------+-----------------+-------------------+---------------+-------------------+---------------+-----------+---------------+
|match_id|              league|match_date|      home_team|home_score|        away_team|away_score|avg_odds_home_win|avg_odds_draw|avg_odds_away_win|max_odds_home_win|max_odds_draw|max_odds_away_win|top_bookie_home_win|top_bookie_draw|top_bookie_away_win|n_odds_home_win|n_odds_draw|n_odds_away_win|
+--------+--------------------+----------+---------------+----------+-----------------+----------+-----------------+-------------+-----------------+-----------------+-------------+-----------------+-------------------+---------------+-------------------+---------------+-----------+---------------+
|  170088|England: Premier ...|2005-01-01|      Liverpool|         0|          Chelsea|         1|     

## Filter Data

Performe a filter operation on a column

In [38]:
df_filtered = df.filter(df.avg_odds_home_win < 2.0)
df_filtered_united = df.filter(df.home_team.contains("United"))

df_filtered.select("league", "home_team", "away_team", "avg_odds_home_win").show(20)
df_filtered_united.select("league", "home_team", "home_score", "away_score", "away_team").show(20)



+--------------------+---------------+--------------+-----------------+
|              league|      home_team|     away_team|avg_odds_home_win|
+--------------------+---------------+--------------+-----------------+
|England: Premier ...|         Fulham|Crystal Palace|           1.9456|
|England: Premier ...|    Aston Villa|     Blackburn|           1.8522|
|England: Premier ...|         Bolton|     West Brom|           1.6122|
|England: Premier ...|Manchester City|   Southampton|              1.7|
|England: Premier ...|  Newcastle Utd|    Birmingham|           1.8622|
|England: Premier ...|     Portsmouth|       Norwich|           1.6344|
|England: Champion...|          Derby|       Cardiff|           1.6875|
|England: Champion...|        Ipswich|      West Ham|           1.8363|
|England: Champion...|          Leeds|         Crewe|           1.8675|
|England: Champion...|            QPR|      Brighton|           1.8388|
|England: Champion...|         Wolves|      Plymouth|           

## Group By and Aggregate

Performe a group by and aggregat operation

In [42]:
from pyspark.sql.functions import count, avg, sum, min, max, desc

df.groupBy("league") \
  .agg(count("*").alias("n_spiele")) \
  .show()

df.groupBy("home_team") \
  .agg(
    sum("home_score").alias("total_home_goals"),
    max("home_score").alias("max_home_goals_in_match")
  ) \
  .orderBy(desc("total_home_goals")) \
  .show(10)



+--------------------+--------+
|              league|n_spiele|
+--------------------+--------+
|Africa: Africa Cu...|     467|
|USA: Atlanta MLS ...|       2|
| Mexico: Copa Mexico|     471|
|Azerbaijan: Super...|       1|
|Belgium: Proximus...|     306|
| Morocco: Botola Pro|     185|
|Netherlands: KNVB...|     520|
|Germany: Regional...|    2010|
|    Ecuador: Serie A|    2218|
|Bulgaria: Bulgari...|     244|
|South Africa: Fir...|     409|
|Germany: Oberliga...|      36|
|Australia: Capita...|      74|
|Slovakia: Fortuna...|     198|
|Germany: Oberliga...|      18|
|Germany: Oberliga...|      32|
|Sweden: Super Cup...|       3|
|FYR of Macedonia:...|       1|
| Austria: Erste Liga|    1312|
|   Asia: Asian Games|     105|
+--------------------+--------+
only showing top 20 rows



+-----------+----------------+-----------------------+
|  home_team|total_home_goals|max_home_goals_in_match|
+-----------+----------------+-----------------------+
|  Barcelona|             849|                      9|
|    Rangers|             811|                      8|
|Real Madrid|             794|                      9|
|River Plate|             749|                      7|
|      Basel|             691|                      7|
|    Everton|             691|                      7|
|   Salzburg|             677|                      8|
|U. Catolica|             665|                      8|
|    Chelsea|             662|                      8|
|    Benfica|             648|                      8|
+-----------+----------------+-----------------------+
only showing top 10 rows



## Save DataFrame to Parquet

In [43]:
df.write \
  .mode("overwrite") \
  .parquet("./closing_odds_parquet")


In [44]:
spark.stop()